In [1]:
import torch
import numpy as np
torch.manual_seed(0)

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986]], requires_grad=True)
tensor([0.4033, 0.8380], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [6]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  -0.4516,  -90.4691],
        [ -24.6303, -132.3828],
        [ -31.2192, -176.1530],
        [  64.3523,  -39.5645],
        [ -73.9524, -161.9561]], grad_fn=<AddBackward0>)


In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


There is a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [9]:
diff = targets - preds
diff

tensor([[ 56.4516, 160.4691],
        [105.6303, 233.3828],
        [150.2192, 309.1530],
        [-42.3523,  76.5645],
        [176.9524, 280.9561]], grad_fn=<SubBackward0>)

In [10]:
diff_sqr = diff * diff
diff_sqr

tensor([[ 3186.7852, 25750.3281],
        [11157.7559, 54467.5312],
        [22565.8203, 95575.5703],
        [ 1793.7162,  5862.1191],
        [31312.1387, 78936.3047]], grad_fn=<MulBackward0>)

In [11]:
torch.sum(diff_sqr) / diff.numel()

tensor(33060.8047, grad_fn=<DivBackward0>)

In [12]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [13]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(33060.8047, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [14]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [15]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986]], requires_grad=True)
tensor([[ -6938.4355,  -9674.6758,  -5744.0205],
        [-17408.7871, -20595.9336, -12453.4707]])


In [16]:
# Gradients for bias
print(b)
print(b.grad)

tensor([0.4033, 0.8380], requires_grad=True)
tensor([ -89.3802, -212.1051])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [17]:
w
w.grad

tensor([[ -6938.4355,  -9674.6758,  -5744.0205],
        [-17408.7871, -20595.9336, -12453.4707]])

In [18]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [19]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(33060.8047, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [20]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [21]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  13.5663,  -58.6043],
        [  -6.1255,  -90.4440],
        [  -8.8863, -126.1837],
        [  77.7158,   -8.3414],
        [ -55.8554, -121.4523]], grad_fn=<AddBackward0>)


In [22]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(23432.4883, grad_fn=<DivBackward0>)


In [23]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -5486.0415,  -8100.2427,  -4775.2969],
        [-14114.9053, -17045.4551, -10264.8691]])
tensor([ -72.1170, -173.0051])


In [24]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [25]:
print(w)
print(b)

tensor([[ 1.6652, -0.1157, -2.0736],
        [ 0.8837, -0.7081, -1.1714]], requires_grad=True)
tensor([0.4050, 0.8419], requires_grad=True)


In [26]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(16931.0820, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [27]:
# Train for 1000 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [28]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(68.2814, grad_fn=<DivBackward0>)


In [29]:
# Predictions
preds

tensor([[ 58.0992,  70.9916],
        [ 75.1894,  96.2924],
        [133.1528, 141.8202],
        [ 25.3490,  39.6753],
        [ 87.3011, 110.1359]], grad_fn=<AddBackward0>)

In [30]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [31]:
mse(targets, preds)

tensor(68.2814, grad_fn=<DivBackward0>)

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [32]:
import torch.nn as nn

In [33]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [34]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [35]:
from torch.utils.data import TensorDataset

In [36]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

In [37]:
train_ds[[1,3,5,7]]

(tensor([[ 91.,  88.,  64.],
         [102.,  43.,  37.],
         [ 74.,  66.,  43.],
         [ 88., 134.,  59.]]),
 tensor([[ 81., 101.],
         [ 22.,  37.],
         [ 57.,  69.],
         [118., 132.]]))

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [38]:
from torch.utils.data import DataLoader

In [39]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [40]:
count = 1
for xb, yb in train_dl:
    print("This is the batch {}".format(count))
    print(xb)
    print(yb)
    count += 1

This is the batch 1
tensor([[ 92.,  87.,  64.],
        [ 73.,  66.,  44.],
        [ 73.,  67.,  43.],
        [ 74.,  66.,  43.],
        [102.,  43.,  37.]])
tensor([[ 82., 100.],
        [ 57.,  69.],
        [ 56.,  70.],
        [ 57.,  69.],
        [ 22.,  37.]])
This is the batch 2
tensor([[ 91.,  87.,  65.],
        [103.,  43.,  36.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 88., 134.,  59.]])
tensor([[ 80., 102.],
        [ 20.,  38.],
        [ 21.,  38.],
        [104., 118.],
        [118., 132.]])
This is the batch 3
tensor([[ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 68.,  97.,  70.],
        [ 69.,  96.,  70.],
        [ 87., 135.,  57.]])
tensor([[119., 133.],
        [ 81., 101.],
        [102., 120.],
        [103., 119.],
        [118., 134.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [41]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4034, -0.5407, -0.3370],
        [ 0.4963,  0.2576,  0.2798]], requires_grad=True)
Parameter containing:
tensor([ 0.0304, -0.2960], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [42]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4034, -0.5407, -0.3370],
         [ 0.4963,  0.2576,  0.2798]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0304, -0.2960], requires_grad=True)]

In [43]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -80.1354,   65.2264],
        [-105.8286,   85.4461],
        [-127.0663,   93.6327],
        [ -76.8333,   71.7569],
        [-103.3026,   78.2677],
        [ -79.9980,   65.4651],
        [-105.6249,   85.4683],
        [-127.8067,   94.4088],
        [ -76.9707,   71.5182],
        [-103.2362,   78.0512],
        [ -79.9317,   65.2486],
        [-105.6912,   85.6848],
        [-127.2700,   93.6105],
        [ -76.8997,   71.9734],
        [-103.4399,   78.0290]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [44]:
# Import nn.functional
import torch.nn.functional as F

In [45]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [46]:
loss = loss_fn(preds, targets)
print(loss)

tensor(17070.0977, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [47]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [48]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [49]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 203.1290
Epoch [20/100], Loss: 250.4733
Epoch [30/100], Loss: 340.5463
Epoch [40/100], Loss: 58.5372
Epoch [50/100], Loss: 105.1758
Epoch [60/100], Loss: 80.7081
Epoch [70/100], Loss: 30.0460
Epoch [80/100], Loss: 20.1902
Epoch [90/100], Loss: 17.6107
Epoch [100/100], Loss: 42.2183


Let's generate predictions using our model and verify that they're close to our targets.

In [50]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.2738,  71.7378],
        [ 80.5657,  98.7157],
        [119.1420, 134.6657],
        [ 28.4415,  45.6655],
        [ 94.8860, 110.6626],
        [ 57.1756,  70.8020],
        [ 80.0847,  98.3978],
        [119.3015, 135.0952],
        [ 29.5397,  46.6013],
        [ 95.5033, 111.2805],
        [ 57.7928,  71.4199],
        [ 79.4674,  97.7799],
        [119.6230, 134.9836],
        [ 27.8242,  45.0477],
        [ 95.9843, 111.5984]], grad_fn=<AddmmBackward>)

In [51]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [52]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.7269, 68.7066]], grad_fn=<AddmmBackward>)

The predicted yield of apples is 54.7 tons per hectare, and that of oranges is 68.7 tons per hectare.